<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_NLP_Project/blob/main/%5BNLP_08%5DCreate_a_Korean_English_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [NLP_08]Create a Korean-English translator

## 1. import

In [ ]:
import urllib.request
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

## 2. 데이터 다운로드

In [ ]:
def data_load(path):
  with open(path, "r") as f:
    raw = f.read().splitlines()
  print('Data Soze : '. len(raw))
  print('Exmaple:')
  for sen in raw[0:100][::20]: print(">>", sen)
  return raw

In [ ]:
path_train_ko = '/content/drive/MyDrive/인공지능/아이펠/GoingDeeper/제출용/data/translator_ke/korean-english-park.train.ko'
path_train_en = '/content/drive/MyDrive/인공지능/아이펠/GoingDeeper/제출용/data/translator_ke/korean-english-park.train.en'

In [ ]:
data_load(path_train_en)

AttributeError: ignored

## 3. 데이터 정제

## 4. 데이터 토큰화

## 5. 모델 설계

## 6. 훈련하기